In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
# from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
# import os, sys

# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [14]:
data = pd.read_csv('02_Data/retail_train.csv')
item_features = pd.read_csv('02_Data/product.csv')
user_features = pd.read_csv('02_Data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [5]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [6]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [7]:
user_features['income_desc'].unique()

array(['35-49K', '50-74K', '25-34K', '75-99K', 'Under 15K', '100-124K',
       '15-24K', '125-149K', '150-174K', '250K+', '175-199K', '200-249K'],
      dtype=object)

In [8]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

In [9]:
user_features['homeowner_desc'].unique()

array(['Homeowner', 'Unknown', 'Renter', 'Probable Renter',
       'Probable Owner'], dtype=object)

In [10]:
user_features['hh_comp_desc'].unique()

array(['2 Adults No Kids', '2 Adults Kids', 'Single Female', 'Unknown',
       'Single Male', '1 Adult Kids'], dtype=object)

In [11]:
user_features['kid_category_desc'].unique()

array(['None/Unknown', '1', '2', '3+'], dtype=object)

## 1. Filter items

In [15]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train.copy(), take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare CSR train matrix

In [20]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(2)

item_id,42346,43020,108646,117847,244960,279994,372396,818981,818996,819048,...,15972565,15972790,16053142,16100266,16729296,16729299,16729363,16729415,16770156,16809649
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [22]:
data_test_filtered = data_test[data_test['item_id'].isin(data_train_filtered['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0)

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_test_user_item = csr_matrix(test_user_item_matrix)

In [128]:
test_user_item_matrix.head(2)

item_id,818981,818996,819210,819487,819590,819969,820011,820301,820352,820486,...,15972298,15972565,15972790,16053142,16100266,16729296,16729299,16729415,16770156,16809649
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [26]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
user_feat.shape

(2499, 7)

In [28]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
42346,69.0,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA
43020,69.0,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA


In [29]:
item_feat.shape

(5001, 6)

## Encoding features

In [30]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [31]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
item_feat_lightfm.head(2)

,manufacturer_2.0,manufacturer_5.0,manufacturer_16.0,manufacturer_20.0,manufacturer_26.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_40.0,manufacturer_42.0,...,curr_size_of_product_L 24 OZ,curr_size_of_product_L16 5/8 OZ,curr_size_of_product_LB,curr_size_of_product_LITER,curr_size_of_product_N 16 OZ,curr_size_of_product_N 40 OZ,curr_size_of_product_PK,curr_size_of_product_PT,curr_size_of_product_QT,curr_size_of_product_SOFT
item_id,,,,,,,,,,,,,,,,,,,,,
42346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [33]:
model = LightFM(no_components=40,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

## Train

In [118]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values),
          item_features=csr_matrix(item_feat_lightfm.values),
          epochs=15, 
          num_threads=4,
          verbose=False) 

# Evaluation

In [37]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values))

In [41]:
user_emb[0]  # biases

array([-10.120368 ,   0.       ,   0.       , ..., -10.522849 ,
        -7.4741707,   0.       ], dtype=float32)

In [42]:
user_emb[0].shape

(2499,)

In [43]:
user_emb[1]  # embeddings

array([[ 5.78285331e-09, -7.17435888e-10,  1.05816742e-08, ...,
         1.19194157e-08,  5.51901636e-09, -9.64732383e-09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 5.75388892e-09, -9.96660421e-10,  1.04517177e-08, ...,
         1.25214035e-08,  5.45326229e-09, -9.24818266e-09],
       [ 9.35261979e-10, -4.88781016e-10,  2.96739078e-09, ...,
         6.89282897e-09,  2.51045185e-09, -2.47420040e-09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [44]:
user_emb[1].shape

(2499, 40)

In [47]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values))

In [48]:
item_emb[0].shape

(5001,)

In [49]:
item_emb[1].shape

(5001, 40)

# Test precision

In [123]:
result = data_test_filtered.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[951954, 971585, 1004906, 1042083, 1131115, 82..."
1,3,[920626]
2,6,"[1104227, 6979393, 13382461, 9859082, 12731467..."
3,7,"[993838, 1108844, 1110779, 10285022, 10285186,..."
4,8,"[13071586, 858001, 905690, 937573, 939275, 946..."
5,9,"[864335, 1029743, 10457112, 889692]"
6,13,"[886317, 1029743, 7409999, 923005, 13133916, 1..."
7,14,"[1029743, 16770156, 6979089, 902377, 1123106]"
8,15,"[13073175, 1082269]"
9,17,"[1053016, 1079541, 9487497, 7409941]"


In [66]:
test_2_item_ids = user_item_matrix.columns.tolist()
test_2_item_ids.remove(999999)
test_2_item_ids = np.array(test_2_item_ids)
test_2_item_ids

array([   42346,    43020,   108646, ..., 16729415, 16770156, 16809649])

In [97]:
def get_recommendations(user, model, N=5):
    predictions = model.predict(user_ids=int(userid_to_id[user]),
                                item_ids=[itemid_to_id[idx] for idx in test_2_item_ids],
                                user_features=csr_matrix(user_feat_lightfm.values),
                                item_features=csr_matrix(item_feat_lightfm.values),
                                num_threads=4)
    recs = [(id_to_itemid[idx], predictions[idx]) for idx in range(len(predictions))]
    recs.sort(key=lambda x: -x[1])
    recs = [itemid for itemid, _ in recs[:N]]
    
    return recs

In [107]:
get_recommendations(3, model, N=5)

[866211, 878996, 854852, 1080989, 1127732]

In [102]:
%%time
    
result['warp'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['warp'], row['actual']), axis=1).mean()

CPU times: user 3min 21s, sys: 20.2 ms, total: 3min 21s
Wall time: 2min 23s


0.04257921067259588

In [104]:
result.head(10)

,user_id,actual,warp
0,1,"[951954, 971585, 1004906, 1042083, 1131115, 82...","[866211, 878996, 854852, 1080989, 1127732]"
1,3,[920626],"[866211, 878996, 854852, 1080989, 1127732]"
2,6,"[1104227, 6979393, 13382461, 9859082, 12731467...","[866211, 878996, 854852, 1080989, 1127732]"
3,7,"[993838, 1108844, 1110779, 10285022, 10285186,...","[866211, 878996, 854852, 1080989, 1127732]"
4,8,"[13071586, 858001, 905690, 937573, 939275, 946...","[866211, 878996, 854852, 1080989, 1127732]"
5,9,"[864335, 1029743, 10457112, 889692]","[866211, 878996, 854852, 1080989, 1127732]"
6,13,"[886317, 1029743, 7409999, 923005, 13133916, 1...","[866211, 878996, 854852, 1080989, 1127732]"
7,14,"[1029743, 16770156, 6979089, 902377, 1123106]","[866211, 878996, 854852, 1080989, 1127732]"
8,15,"[13073175, 1082269]","[866211, 878996, 854852, 1080989, 1127732]"
9,17,"[1053016, 1079541, 9487497, 7409941]","[866211, 878996, 854852, 1080989, 1127732]"
